# Entrenamiento del modelo

In [1]:
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

# Definir las transformaciones para los conjuntos de datos
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = 'directorio_datos'  # Asegúrate de reemplazar esto con la ruta a tu conjunto de datos
batch_size = 32

image_datasets = {
    x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
    for x in ['train', 'val']
}
dataloaders = {
    x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4)
    for x in ['train', 'val']
}


In [ ]:
from torch import nn

model = models.resnet101(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Ajustar para # clases

import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [ ]:
num_epochs = 12  

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in dataloaders['train']:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects.double() / len(image_datasets['train'])

    print(f'Epoch {epoch}/{num_epochs - 1}, Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}')


# Guardado del modelo

In [5]:
torch.save(model.state_dict(), 'resnet101_model.pth')


# Matriz de confusion y recall

In [ ]:
import torch
from torchvision import models
from torch import nn

# Define el modelo (asegúrate de que sea idéntico al modelo que entrenaste)
model = models.resnet101(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 5)  # Ajusta para 5 clases

# Cargar los pesos del modelo guardado
model.load_state_dict(torch.load('resnet101_model.pth'))

model.eval()  # Establece el modelo en modo de evaluación

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

# Define las transformaciones (deberían ser las mismas que usaste para la validación durante el entrenamiento)
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Cargar el conjunto de datos de test
test_dataset = datasets.ImageFolder(root=os.path.join('directorio_datos', 'test'), transform=transform)
test_dataset_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

y_pred = []
y_true = []

# Desactiva el cálculo de gradientes para ahorrar memoria y acelerar el proceso
with torch.no_grad():
    for inputs, labels in test_dataset_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_pred.extend(preds.view(-1).cpu().numpy())
        y_true.extend(labels.view(-1).cpu().numpy())
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Calcular la matriz de confusión
conf_mat = confusion_matrix(y_true, y_pred)
recall = recall_score(y_true, y_pred, average='macro')

print(f'Recall del modelo: {recall}')
# Mostrar la matriz de confusión
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(conf_mat, annot=True, fmt='d',
            xticklabels=test_dataset.classes, yticklabels=test_dataset.classes)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.title('Matriz de Confusión')
plt.show()
